### Import packages and standardize data

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from scipy.stats import pearsonr
from sklearn.frozen import FrozenEstimator
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import roc_auc_score
import seaborn as sns
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import xgboost as xgb
from xgboost import XGBClassifier
import multiprocessing
import logging
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [3]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print(torch.__path__ )



Using device: cuda
['/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/torch']


In [4]:
### Update with joined geno-pheno data

chunksize = 100

list_of_dataframes = []

for df in pd.read_csv('DarpaQCGenoPheno.csv', chunksize=chunksize, index_col=0):
    list_of_dataframes.append(df)

result = pd.concat(list_of_dataframes)
df = result
df

ID  AX-574114010  AX-564298109  AX-574114011  AX-577073921  \
0     B-1000            -1            -1            -1            -1   
1     B-1002            -1            -1            -1             1   
2     B-1003            -1            -1             0            -1   
3     B-1005            -1            -1             1             1   
4     B-1006            -1            -1             1             1   
...      ...           ...           ...           ...           ...   
2352   Y_988            -1            -1             0             0   
2353   Y_991             0             0            -1            -1   
2354   Y_992            -1            -1             0             1   
2355   Y_995            -1            -1            -1            -1   
2356   Y_998             1             1            -1            -1   

      AX-564298209  AX-574114024  AX-576891682  AX-564298228  AX-576891683  \
0               -1            -1            -1             1             1   
1                1             1             1             1             1   
2                0             0             0             0             0   
3                0             0             0             0             0   
4                1             1             1             1             1   
...            ...           ...           ...           ...           ...   
2352            -1            -1             0             0             0   
2353            -1            -1            -1             1             1   
2354            -1             0             0             0             0   
2355            -1            -1            -1             1             1   
2356            -1            -1            -1             1             1   

      ...  AX-568138441  AX-574093181  AX-574093191  AX-574093227       Pop  \
0     ...             0            -1             0             0  Training   
1     ...             0             0            -1             0  Training   
2     ...             0            -1            -1             1  Training   
3     ...             1            -1             0             1  Training   
4     ...             0            -1            -1             1  Training   
...   ...           ...           ...           ...           ...       ...   
2352  ...            -1             0             0             1  Training   
2353  ...            -1            -1             0             1  Training   
2354  ...             1             0             0             0  Training   
2355  ...             0             0             0             1  Training   
2356  ...             1             1            -1             0  Training   

      Plate  Status    WWt   Group  Generation  
0      RU11       0    NaN      F0          F0  
1      RU10       0    NaN      F0          F0  
2      RU10       0    NaN      F0          F0  
3      RU11       0    NaN      F0          F0  
4      RU10       1    NaN      F0          F0  
...     ...     ...    ...     ...         ...  
2352   RU28       1  12.73  FLGS23          F2  
2353   RU28       1  11.30  FLGS23          F2  
2354   RU28       0   9.48  FLGS23          F2  
2355   RU28       1   7.81  FLGS23          F2  
2356   RU28       1  10.54  FLGS23          F2  

[2357 rows x 37022 columns]

In [5]:
ax_columns = [col for col in df.columns if col.startswith('AX')]
# len(ax_columns)
X = df[ax_columns]
y = df["Status"]
X = X.to_numpy()
y = y.to_numpy()

scaler = StandardScaler()
X = scaler.fit_transform(X)
X

array([[-0.81869812, -0.76517567, -0.94206249, ..., -1.01229399,
         1.03438912, -1.87421104],
       [-0.81869812, -0.76517567, -0.94206249, ...,  0.40793037,
        -0.76491358, -1.87421104],
       [-0.81869812, -0.76517567,  0.58611458, ..., -1.01229399,
        -0.76491358,  0.43017885],
       ...,
       [-0.81869812, -0.76517567,  0.58611458, ...,  0.40793037,
         1.03438912, -1.87421104],
       [-0.81869812, -0.76517567, -0.94206249, ...,  0.40793037,
         1.03438912,  0.43017885],
       [ 1.8782571 ,  2.16499337, -0.94206249, ...,  1.82815473,
        -0.76491358, -1.87421104]])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

## XGB hp tuning

In [14]:
class ProgressPrinter:
    def __init__(self, total_iter):
        self.total_iter = total_iter
        self.current_iter = 0

    def __call__(self, optim_result):
        self.current_iter += 1
        print(f"Iteration {self.current_iter}/{self.total_iter} finished")
        return False  # continue optimization
        
if __name__ == "__main__":
    print("Starting Bayesian optimization for XGBoost HP tuning")

    clf = XGBClassifier(
        random_state=8,
        tree_method="hist",  # efficient with many features
        n_jobs=multiprocessing.cpu_count() // 2,
        use_label_encoder=False,
        eval_metric="logloss"
    )

    # Param search space
    search_space = {
        'max_depth': Integer(2, 8),
        'learning_rate': Real(0.001, 1.0, prior='log-uniform'),
        'subsample': Real(0.5, 1.0),
        'colsample_bytree': Real(0.5, 1.0),
        'colsample_bylevel': Real(0.5, 1.0),
        'colsample_bynode': Real(0.5, 1.0),
        'reg_alpha': Real(0.0, 10.0),
        'reg_lambda': Real(0.0, 10.0),
        'gamma': Real(0.0, 10.0),
        'n_estimators': Integer(100, 1000)
    }


    # 3-fold CV with Bayesian optimization
    opt = BayesSearchCV(
        estimator=clf,
        cv=3,
        search_spaces=search_space,
        n_iter=25, #adjust as needed
        scoring='roc_auc',
        n_jobs=2,
        random_state=8,
        verbose=0  
    )

    print("Fitting BayesSearchCV...")
    opt.fit(X_train, y_train, callback=[ProgressPrinter(n_iter)])

    print(f"Best cross-validated AUC: {opt.best_score_:.4f}")
    print(f"Best parameters: {opt.best_params_}")

    # Evaluate on held-out test set
    y_pred_proba = opt.best_estimator_.predict_proba(X_test)[:, 1]
    test_auc = roc_auc_score(y_test, y_pred_proba)
    print(f"Test AUC: {test_auc:.4f}")

    best_model = opt.best_estimator_


Starting Bayesian optimization for XGBoost HP tuning
Fitting BayesSearchCV...


/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [11:07:29] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [11:07:29] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [11:09:40] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Iteration 1/50 finished


/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [11:11:41] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [11:11:41] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [11:14:06] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Iteration 2/50 finished


...

Iteration 24/50 finished


/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:45:41] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:45:45] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:46:10] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Iteration 25/50 finished


/hpc/group/schultzlab/hs325/miniconda3/envs/gsAI/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [12:46:30] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best cross-validated AUC: 0.6074
Best parameters: OrderedDict({'colsample_bylevel': 0.5331080642783614, 'colsample_bynode': 0.7262320733948379, 'colsample_bytree': 0.5362052518135637, 'gamma': 7.436578137661654, 'learning_rate': 0.0011002938624638172, 'max_depth': 5, 'n_estimators': 551, 'reg_alpha': 1.964788283871643, 'reg_lambda': 0.5172779921798499, 'subsample': 0.8441395984251803})
Test AUC: 0.6726


In [ ]:
## now: test best model 10x StratifiedKFold
'''
Best cross-validated AUC: 0.6074
Best parameters: OrderedDict({'colsample_bylevel': 0.5331080642783614, 'colsample_bynode': 0.7262320733948379, 'colsample_bytree': 0.5362052518135637, 'gamma': 7.436578137661654, 'learning_rate': 0.0011002938624638172, 'max_depth': 5, 'n_estimators': 551, 'reg_alpha': 1.964788283871643, 'reg_lambda': 0.5172779921798499, 'subsample': 0.8441395984251803})
Test AUC: 0.6726
'''
